# Analyzing Toronto Neighborhoods 

## Segmenting and Clustering Neighborhoods in Toronto

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

# Part I

* Start by creating a new Notebook for this assignment.  
* Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:  
* To create the above dataframe:

* The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
* Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
* More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
* If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
* Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
* In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.                

Importing libs

In [1]:
import json
import pandas as pd
from bs4 import BeautifulSoup
import requests

Loading the dataframe from Wiki page

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)

Parsing the HTLM and getting the table

In [3]:
soup = BeautifulSoup(page.text, 'html.parser')
table = soup.find('table');

Parsing the table assigning it to a DataFrame and setting column names

In [4]:
table_rows = table.find_all('tr')
l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)
df = pd.DataFrame(l, columns = ['PostalCode', 'Borough', 'Neighborhood'])
df.head()

,PostalCode,Borough,Neighborhood
0,None,None,None
1,M1A,Not assigned,Not assigned\n
2,M2A,Not assigned,Not assigned\n
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n


Data Cleanup
* Removing \n at the end of Neighborhood names
* Removing first row with Nones

In [5]:
df['Neighborhood'] = df['Neighborhood'].str.rstrip('\n')
df.drop(0, inplace = True)
df.sort_values(by = 'PostalCode', inplace = True)
df.head()

,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
13,M1B,Scarborough,Malvern
12,M1B,Scarborough,Rouge
30,M1C,Scarborough,Port Union
29,M1C,Scarborough,Rouge Hill


Remove the 'Not assigned' values from Borough column

In [6]:
df.drop(df[df['Borough'] == 'Not assigned'].index, inplace = True)
df.head()

,PostalCode,Borough,Neighborhood
13,M1B,Scarborough,Malvern
12,M1B,Scarborough,Rouge
30,M1C,Scarborough,Port Union
29,M1C,Scarborough,Rouge Hill
28,M1C,Scarborough,Highland Creek


Merging rows with duplicate PostalCode and Borough, retain Neighborhood values and separate by comma

In [7]:
# Getting the Boroughs
df_boroughs = df[['PostalCode', 'Borough']].copy()
df_boroughs.drop_duplicates(keep = 'first', inplace = True)
df_boroughs.reset_index();

In [8]:
# Group Neighborhood by PostalCode
df_grouped = df.groupby('PostalCode')['Neighborhood'].apply(', '.join).reset_index()
df_grouped.head()

,PostalCode,Neighborhood
0,M1B,"Malvern, Rouge"
1,M1C,"Port Union, Rouge Hill, Highland Creek"
2,M1E,"West Hill, Morningside, Guildwood"
3,M1G,Woburn
4,M1H,Cedarbrae


In [9]:
# Merging in the Boroughs again
df_new = pd.merge(df_boroughs, df_grouped, on = 'PostalCode')
df_new.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek"
2,M1E,Scarborough,"West Hill, Morningside, Guildwood"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


If Neighborhood is 'Not assigned' set it to value of Borough and print Postal Code 'M7A' for Queen's Park to validate

In [10]:
df_new.loc[df_new.loc[df_new['Neighborhood'] == 'Not assigned'].index,'Neighborhood'] = \
    df_new.loc[df_new['Neighborhood'] == 'Not assigned']['Borough'] 
df_new.head(86)
# Check the last printed record (85) for Queen's Park being the value of both Borough and Neighborhood Columns

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek"
2,M1E,Scarborough,"West Hill, Morningside, Guildwood"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Oakridge, Golden Mile, Clairlea"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Cliffside West, Birch Cliff"


Using the .shape method to print the number of rows of dataframe.

In [11]:
print(df_new.shape)

(103, 3)
